# Data Acquisition

In [5]:
import requests
from time import sleep

def fetch_data(api_url):
    try:
        response = requests.get(api_url)
        # if status is 429 (Too Many Requests)
        if response.status_code == 429:
            print("Rate limit exceeded. Waiting before retrying...")
            # Extract the retry-after time
            retry_after = int(response.headers.get('Retry-After', 10))
            sleep(retry_after)
            return fetch_data(api_url)  # Retry fetching data after waiting

        response.raise_for_status()  # Check for other HTTP errors
        return response.json()
    
    except requests.exceptions.HTTPError as http_err:
        if response.status_code == 404:
            print("Resource not found (404). Check the URL or endpoint.")
        elif response.status_code == 500:
            print("Server error (500). Try again later.")
        else:
            print(f"HTTP error occurred: {http_err}")
    
    except requests.exceptions.RequestException as req_err:
        print(f"A request exception occurred: {req_err}")
    
    except Exception as err:
        print(f"An error occurred: {err}")

    return None

In [6]:
# key = demo
api_url = "https://api.opensustainabilityindex.org/v1/companies?api-key=demo"

# Fetch data
data = fetch_data(api_url)
if data is None:
    # Handle missing data scenario
    print("Failed to fetch data. Retrying after a short wait...")
    sleep(10)
    data = fetch_data(api_url)

# Check
if data is not None:
    print("Data fetched successfully.")
    print(data)
else:
    print("Failed to fetch data after retrying.")


Data fetched successfully.
{'data': [{'company_name': '3i Group plc', 'slug': '3i', 'company_url': None, 'industry': 'Banks, Diverse Financials, Insurance', 'organization_type': 'Financial Institution', 'tags': None, 'commitment_type': 'Standard', 'status': 'Active', 'commitment_deadline': '2025-04-30', 'total_reported_emission_scope_1_2_3': 6983.9, 'revenue': 5804.04, 'hq_country': 'United Kingdom', 'year': 2023, 'currency': 'USD', 'emission_intensity': 1.2, 'targets': [{'target_type': None, 'target_scope': None, 'target_year': None, 'target': None}]}, {'company_name': '3M', 'slug': '3m', 'company_url': '3m.com', 'industry': 'Chemicals', 'organization_type': 'Company', 'tags': None, 'commitment_type': 'Standard', 'status': 'Active', 'commitment_deadline': '2025-11-30', 'total_reported_emission_scope_1_2_3': 15516000, 'revenue': 34200, 'hq_country': 'United States of America', 'year': 2022, 'currency': 'USD', 'emission_intensity': 453.68, 'targets': [{'target_type': None, 'target_scope

# Data Cleaning

## 1. Companies (Get companies)

In [44]:
import requests
import pandas as pd

# Replace with your actual API key
api_key = "demo"
url = "https://api.opensustainabilityindex.org/v1/companies?api-key=demo"

response = requests.get(url)
data = response.json()

# Check the structure of the data
print(data)

{'data': [{'company_name': '3i Group plc', 'slug': '3i', 'company_url': None, 'industry': 'Banks, Diverse Financials, Insurance', 'organization_type': 'Financial Institution', 'tags': None, 'commitment_type': 'Standard', 'status': 'Active', 'commitment_deadline': '2025-04-30', 'total_reported_emission_scope_1_2_3': 6983.9, 'revenue': 5804.04, 'hq_country': 'United Kingdom', 'year': 2023, 'currency': 'USD', 'emission_intensity': 1.2, 'targets': [{'target_type': None, 'target_scope': None, 'target_year': None, 'target': None}]}, {'company_name': '3M', 'slug': '3m', 'company_url': '3m.com', 'industry': 'Chemicals', 'organization_type': 'Company', 'tags': None, 'commitment_type': 'Standard', 'status': 'Active', 'commitment_deadline': '2025-11-30', 'total_reported_emission_scope_1_2_3': 15516000, 'revenue': 34200, 'hq_country': 'United States of America', 'year': 2022, 'currency': 'USD', 'emission_intensity': 453.68, 'targets': [{'target_type': None, 'target_scope': None, 'target_year': Non

In [46]:
# Convert 'data' part to DataFrame
df_company = pd.DataFrame(data['data'])

# Display the first few rows to understand the structure
print(df_company.head())
# Check for missing values
print(df_company.isnull().sum())

# Handle missing values
df_company = df_company.fillna({
    'company_name': 'Unknown',
    'industry': 'Unknown',
    'commitment_type': 'Unknown',
    'status': 'Unknown',
    'commitment_deadline': 'Unknown',
    'total_reported_emission_scope_1_2_3': 0,
    'revenue': 0,
    'hq_country_move': 'Unknown',
    'year': 0,
    'currency': 'Unknown',
    'emission_intensity': 0
})
# Expand 'targets' list into a separate DataFrame
targets_df = pd.json_normalize(df_company['targets'].explode())

# Display the targets DataFrame
print(targets_df.head())

# Remove the 'targets' column from the main DataFrame and merge with the targets DataFrame if needed
df_company = df_company.drop(columns=['targets'])


               company_name              slug  company_url  \
0              3i Group plc                3i         None   
1                        3M                3m       3m.com   
2  A.P. Moller - Maersk A/S  ap-moller-maersk   maersk.com   
3                       AAK               aak         None   
4         AB Anders Löfberg    anders-lofberg  lofbergs.se   

                                      industry      organization_type    tags  \
0         Banks, Diverse Financials, Insurance  Financial Institution    None   
1                                    Chemicals                Company    None   
2  Water Transportation - Water Transportation                Company  omx120   
3                 Food and Beverage Processing                Company  omx120   
4                 Food and Beverage Processing                    SME    None   

  commitment_type      status commitment_deadline  \
0        Standard      Active          2025-04-30   
1        Standard      Active     

In [49]:
# Remove duplicate rows
df_cleaned = df_company.drop_duplicates()

# Convert numerical columns to appropriate types
df_cleaned['total_reported_emission_scope_1_2_3'] = df_cleaned['total_reported_emission_scope_1_2_3'].astype(float)
df_cleaned['revenue'] = df_cleaned['revenue'].astype(float)
df_cleaned['year'] = df_cleaned['year'].astype(int)
df_cleaned['emission_intensity'] = df_cleaned['emission_intensity'].astype(float)

# Ensure text fields are strings
df_cleaned['company_name'] = df_cleaned['company_name'].astype(str)
df_cleaned['industry'] = df_cleaned['industry'].astype(str)
df_cleaned['commitment_type'] = df_cleaned['commitment_type'].astype(str)
df_cleaned['status'] = df_cleaned['status'].astype(str)
df_cleaned['commitment_deadline'] = df_cleaned['commitment_deadline'].astype(str)
df_cleaned['currency'] = df_cleaned['currency'].astype(str)

# Convert to lowercase and strip leading/trailing whitespace
df_cleaned['company_name'] = df_cleaned['company_name'].str.lower().str.strip()
df_cleaned['industry'] = df_cleaned['industry'].str.lower().str.strip()
df_cleaned['commitment_type'] = df_cleaned['commitment_type'].str.lower().str.strip()
df_cleaned['status'] = df_cleaned['status'].str.lower().str.strip()
df_cleaned['commitment_deadline'] = df_cleaned['commitment_deadline'].str.lower().str.strip()
df_cleaned['currency'] = df_cleaned['currency'].str.lower().str.strip()

# Convert 'commitment_deadline' to datetime if it contains dates
df_cleaned['commitment_deadline'] = pd.to_datetime(df_cleaned['commitment_deadline'], errors='coerce')

# Save to a CSV file
df_cleaned.to_csv('cleaned_companies_data.csv', index=False)

# Display the cleaned data
print(df_cleaned.head())

               company_name              slug  company_url  \
0              3i group plc                3i         None   
1                        3m                3m       3m.com   
2  a.p. moller - maersk a/s  ap-moller-maersk   maersk.com   
3                       aak               aak         None   
4         ab anders löfberg    anders-lofberg  lofbergs.se   

                                      industry      organization_type    tags  \
0         banks, diverse financials, insurance  Financial Institution    None   
1                                    chemicals                Company    None   
2  water transportation - water transportation                Company  omx120   
3                 food and beverage processing                Company  omx120   
4                 food and beverage processing                    SME    None   

  commitment_type      status commitment_deadline  \
0        standard      active          2025-04-30   
1        standard      active     

## 1 Companies (Get details of a specific company by its slug)

In [19]:
import requests

def fetch_company_details(slug, api_key='demo'):
    api_url = f'https://api.opensustainabilityindex.org/v1/companies/{slug}?api-key={api_key}'
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Check for HTTP errors
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")
    return None

# Fetch company details
company_slug = 'commonshare'  # Replace with the actual company slug
company_data = fetch_company_details(company_slug)

if company_data:
    print("Company Data:", company_data)  # Inspect the data structure
else:
    print("No data returned or error occurred.")

Company Data: {'data': {}, 'error': {'code': 'PGRST116', 'details': 'The result contains 0 rows', 'hint': None, 'message': 'JSON object requested, multiple (or no) rows returned'}}


In [23]:
import requests
import pandas as pd
import numpy as np

def fetch_company_details(company_slug, api_key):
    url = f"https://api.opensustainabilityindex.org/v1/companies/{company_slug}?api-key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json().get('data', {})
        print(data)  # Print the fetched data to inspect its structure
        return data
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

def flatten_company_details(data):
    # Flatten emissions list
    emissions = data.get('emissions', [])
    if not isinstance(emissions, list):
        emissions = []
    emissions_df = pd.json_normalize(emissions)
    
    # Flatten targets list
    targets = data.get('targets', [])
    if not isinstance(targets, list):
        targets = []
    targets_df = pd.json_normalize(targets)
    
    # Flatten commitment dictionary
    commitment = data.get('commitment', [])
    if not isinstance(commitment, list):
        commitment = []
    commitment_df = pd.json_normalize(commitment)

    # Create a DataFrame for the main data
    main_df = pd.DataFrame({
        'company_name': data.get('company_name', 'Unknown'),
        'industry': data.get('industry', 'Unknown'),
        'isic': data.get('isic', 'Unknown'),
        'lei': data.get('lei', 'Unknown'),
        'company_url': data.get('company_url', 'Unknown'),
        'source_reports_page': data.get('source_reports_page', 'Unknown'),
        'sbt_status': data.get('sbt_status', 'Unknown'),
        'sbt_near_term_year': data.get('sbt_near_term_year', 'Unknown'),
        'sbt_near_term_target': data.get('sbt_near_term_target', 'Unknown'),
        'net_zero_year': data.get('net_zero_year', 'Unknown'),
        'slug': data.get('slug', 'Unknown')
    }, index=[0])
    
    # Initialize cleaned_df
    cleaned_df = main_df.copy()
    
    # Join all data if DataFrames are not empty
    if not emissions_df.empty:
        cleaned_df = cleaned_df.join(emissions_df, how='left')
    if not targets_df.empty:
        cleaned_df = cleaned_df.join(targets_df, how='left', rsuffix='_target')
    if not commitment_df.empty:
        cleaned_df = cleaned_df.join(commitment_df, how='left', rsuffix='_commitment')

    return cleaned_df

def clean_company_details_data(df):
    # Handle missing values
    df.fillna({
        'company_name': 'Unknown',
        'industry': 'Unknown',
        'isic': 'Unknown',
        'lei': 'Unknown',
        'company_url': 'Unknown',
        'source_reports_page': 'Unknown',
        'sbt_status': 'Unknown',
        'sbt_near_term_year': 'Unknown',
        'sbt_near_term_target': 'Unknown',
        'net_zero_year': 'Unknown',
        'slug': 'Unknown'
    }, inplace=True)
    
    # Remove duplicates
    df.drop_duplicates(inplace=True)
    
    # Convert numerical columns to appropriate types
    numerical_cols = ['total_scope_3', 'cradle_to_gate', 'scope_2_location_based', 'total_reported_emission_scope_1_2', 'total_reported_emission_scope_1_2_3']
    for col in numerical_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Ensure 'year' columns are integers
    year_cols = ['target_year']
    for col in year_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
    
    return df

# Replace with actual company slug and API key
company_slug = '3i'
api_key = 'demo'

# Fetch data
company_data = fetch_company_details(company_slug, api_key)

if company_data:
    # Flatten and clean the data
    df = flatten_company_details(company_data)
    cleaned_df = clean_company_details_data(df)
    print(cleaned_df.head())  # Display the first few rows of the cleaned data
else:
    print("No data fetched.")

{'company_name': '3i Group plc', 'name': '3i Group plc', 'slug_old': '3i-group-plc', 'slug': '3i', 'company_url': None, 'industry': 'Banks, Diverse Financials, Insurance', 'isic': None, 'lei': '35GDVHRBMFE7NWATNM84', 'emissions': [{'year': 2022, 'cat_1': 2418.7, 'cat_2': None, 'cat_3': 45.1, 'cat_4': None, 'cat_5': 16.4, 'cat_6': 322.2, 'cat_7': 147.9, 'cat_8': None, 'cat_9': None, 'cat_10': None, 'cat_11': None, 'cat_12': None, 'cat_13': None, 'cat_14': None, 'cat_15': None, 'status': None, 'revenue': 5045.32, 'scope_1': 130, 'all_cats': 'Partly', 'currency': 'USD', 'created_at': '2024-06-07T08:42:26.772362+00:00', 'updated_at': '2024-06-07T08:42:26.772362+00:00', 'fiscal_year': '2022', 'company_slug': '3i', 'ghg_standard': None, 'page_revenue': '162', 'emission_page': '64', 'revenue_local': 4079, 'total_scope_3': 2950, 'cradle_to_gate': 0.62, 'currency_local': 'GBP', 'source_revenue': 'https://www.3i.com/media/orvbwrof/3i_group_ar23.pdf', 'scope_2_unknown': None, 'publication_date': 

## 2. Industries

In [40]:
import requests
import pandas as pd

# Replace with your actual API key
api_key = "demo"
url = f"https://api.opensustainabilityindex.org/v1/industries?api-key={api_key}"

response = requests.get(url)
data = response.json()

# Check the structure of the data
print(data)

{'data': [{'name': 'Aerospace and Defense', 'slug': 'aerospace-and-defense', 'company_count': 27}, {'name': 'Air Freight Transportation and Logistics', 'slug': 'air-freight-transportation-and-logistics', 'company_count': 79}, {'name': 'Air Transportation - Airlines', 'slug': 'air-transportation---airlines', 'company_count': 29}, {'name': 'Air Transportation - Airport Services', 'slug': 'air-transportation---airport-services', 'company_count': 14}, {'name': 'Automobiles and Components', 'slug': 'automobiles-and-components', 'company_count': 293}, {'name': 'Banks, Diverse Financials, Insurance', 'slug': 'banks-diverse-financials-insurance', 'company_count': 273}, {'name': 'Building Products', 'slug': 'building-products', 'company_count': 169}, {'name': 'Chemicals', 'slug': 'chemicals', 'company_count': 234}, {'name': 'Construction Materials', 'slug': 'construction-materials', 'company_count': 107}, {'name': 'Construction and Engineering', 'slug': 'construction-and-engineering', 'company_

In [41]:
# Convert the 'data' part to a DataFrame
df_industry = pd.DataFrame(data['data'])

# Display the first few rows to understand the structure
print(df_industry.head())
# Check for missing values
print(df_industry.isnull().sum())

# Handle missing values
# If 'company_count' is missing, fill with 0
df_industry = df_industry.fillna({'name': 'Unknown', 'slug': 'unknown-slug', 'company_count': 0})
# Remove duplicate rows
df_cleaned = df_industry.drop_duplicates()
# Convert 'company_count' to integer
df_cleaned['company_count'] = df_cleaned['company_count'].astype(int)

# Ensure text fields are strings
df_cleaned['name'] = df_cleaned['name'].astype(str)
df_cleaned['slug'] = df_cleaned['slug'].astype(str)

                                       name  \
0                     Aerospace and Defense   
1  Air Freight Transportation and Logistics   
2             Air Transportation - Airlines   
3     Air Transportation - Airport Services   
4                Automobiles and Components   

                                       slug  company_count  
0                     aerospace-and-defense             27  
1  air-freight-transportation-and-logistics             79  
2             air-transportation---airlines             29  
3     air-transportation---airport-services             14  
4                automobiles-and-components            293  
name             0
slug             0
company_count    0
dtype: int64


In [42]:
# Convert to lowercase and strip leading/trailing whitespace
df_cleaned['name'] = df_cleaned['name'].str.lower().str.strip()
df_cleaned['slug'] = df_cleaned['slug'].str.lower().str.strip()
# Save to a CSV file
df_cleaned.to_csv('cleaned_industries_data.csv', index=False)

# Display the cleaned data
print(df_cleaned.head())

                                       name  \
0                     aerospace and defense   
1  air freight transportation and logistics   
2             air transportation - airlines   
3     air transportation - airport services   
4                automobiles and components   

                                       slug  company_count  
0                     aerospace-and-defense             27  
1  air-freight-transportation-and-logistics             79  
2             air-transportation---airlines             29  
3     air-transportation---airport-services             14  
4                automobiles-and-components            293  


## 3.Search 

In [35]:
import requests
import pandas as pd

# Replace with your actual API key and query
api_key = "demo"
query = "get"

url = f"https://api.opensustainabilityindex.org/v1/search?api-key={api_key}&query={query}"

response = requests.get(url)
data = response.json()

# Convert the 'data' part to a DataFrame
df_search = pd.DataFrame(data['data'])

# Display the first few rows to understand the structure
print(df_search.head())

                                          name  \
0                 Actioned on Target dashboard   
1                           Energetics Pty Ltd   
2                             GET-IT CO., LTD.   
3                                      GETLINK   
4  Getac Precision Technology Vietnam Co., Ltd   

                                 slug     type  
0        actioned-on-target-dashboard  company  
1                          energetics  company  
2                              get-it  company  
3                             getlink  company  
4  getac-precision-technology-vietnam  company  


In [36]:
df_search = pd.DataFrame(data['data'])
# Check for missing values
print(df.isnull().sum())

# Fill missing values with a placeholder
df_filled = df_search.fillna({'name': 'Unknown', 'slug': 'unknown-slug', 'type': 'unknown-type'})
# Remove duplicate rows
df_cleaned = df_cleaned.drop_duplicates()

# Convert 'type' to lowercase (if case-insensitive comparison is needed)
df_cleaned['type'] = df_cleaned['type'].str.lower()

# Ensure all strings are stripped of leading/trailing spaces
df_cleaned['name'] = df_cleaned['name'].str.strip()
df_cleaned['slug'] = df_cleaned['slug'].str.strip()
df_cleaned['type'] = df_cleaned['type'].str.strip()
# Save to a CSV file
df_cleaned.to_csv('cleaned_data_search.csv', index=False)
print(df_cleaned.head())

name    0
slug    0
type    0
dtype: int64
                                          name  \
0                 Actioned on Target dashboard   
1                           Energetics Pty Ltd   
2                             GET-IT CO., LTD.   
3                                      GETLINK   
4  Getac Precision Technology Vietnam Co., Ltd   

                                 slug     type  
0        actioned-on-target-dashboard  company  
1                          energetics  company  
2                              get-it  company  
3                             getlink  company  
4  getac-precision-technology-vietnam  company  


# Data Transformation

# ETL PIPELINE

In [62]:
import requests
import pandas as pd

class ETLPipeline:
    def __init__(self, api_key):
        self.api_key = api_key

    def extract(self, url):
        response = requests.get(url)
        return response.json()

    def transform_industry(self, data):
        df = pd.DataFrame(data['data'])
        df = df.fillna({'name': 'Unknown', 'slug': 'unknown-slug', 'company_count': 0})
        df = df.drop_duplicates()
        df['company_count'] = df['company_count'].astype(int)
        df['name'] = df['name'].astype(str).str.lower().str.strip()
        df['slug'] = df['slug'].astype(str).str.lower().str.strip()
        return df.groupby('name').agg({'company_count': 'sum'}).reset_index()

    def transform_search(self, data):
        df = pd.DataFrame(data['data'])
        df = df.fillna({'name': 'Unknown', 'slug': 'unknown-slug', 'type': 'unknown-type'})
        df = df.drop_duplicates()
        df['name'] = df['name'].astype(str).str.lower().str.strip()
        df['slug'] = df['slug'].astype(str).str.lower().str.strip()
        df['type'] = df['type'].astype(str).str.lower().str.strip()
        return df.groupby('type').size().reset_index(name='count')

    def transform_companies(self, data):
        df = pd.DataFrame(data['data'])

        # Handle missing values
        df = df.fillna({
            'company_name': 'Unknown',
            'industry': 'Unknown',
            'commitment_type': 'Unknown',
            'status': 'Unknown',
            'commitment_deadline': 'Unknown',
            'total_reported_emission_scope_1_2_3': 0,
            'revenue': 0,
            'hq_country_move': 'Unknown',
            'year': 0,
            'currency': 'Unknown',
            'emission_intensity': 0
        })

        # Expand 'targets' list into a separate DataFrame
        targets_df = pd.json_normalize(df['targets'].explode())

        # Remove 'targets' column from the main DataFrame
        df = df.drop(columns=['targets'])

        # Remove duplicate rows
        df = df.drop_duplicates()

        # Convert numerical columns to appropriate types
        df['total_reported_emission_scope_1_2_3'] = df['total_reported_emission_scope_1_2_3'].astype(float)
        df['revenue'] = df['revenue'].astype(float)
        df['year'] = df['year'].astype(int)
        df['emission_intensity'] = df['emission_intensity'].astype(float)

        # Ensure text fields are strings
        df['company_name'] = df['company_name'].astype(str).str.lower().str.strip()
        df['industry'] = df['industry'].astype(str).str.lower().str.strip()
        df['commitment_type'] = df['commitment_type'].astype(str).str.lower().str.strip()
        df['status'] = df['status'].astype(str).str.lower().str.strip()
        df['commitment_deadline'] = df['commitment_deadline'].astype(str).str.lower().str.strip()
        df['currency'] = df['currency'].astype(str).str.lower().str.strip()

        # Convert 'commitment_deadline' to datetime
        df['commitment_deadline'] = pd.to_datetime(df['commitment_deadline'], errors='coerce')

        # Return both cleaned companies DataFrame and targets DataFrame
        return df, targets_df

    def load(self, df, filename):
        df.to_csv(filename, index=False)
        print(f'Data saved to {filename}')

    def run(self, industry_url, search_url, companies_url):
        # Industry data
        industry_data = self.extract(industry_url)
        transformed_industry = self.transform_industry(industry_data)
        self.load(transformed_industry, 'transformed_industries_summary.csv')
        
        # Search data
        search_data = self.extract(search_url)
        transformed_search = self.transform_search(search_data)
        self.load(transformed_search, 'transformed_search_summary.csv')
        
        # Companies data
        companies_data = self.extract(companies_url)
        transformed_companies, targets_df = self.transform_companies(companies_data)
        self.load(transformed_companies, 'cleaned_companies_data.csv')

        #save targets data to a CSV
        self.load(targets_df, 'cleaned_targets_data.csv')

# Usage
api_key = "demo"
industry_url = f"https://api.opensustainabilityindex.org/v1/industries?api-key={api_key}"
search_url = f"https://api.opensustainabilityindex.org/v1/search?api-key={api_key}&query=get"
companies_url = f"https://api.opensustainabilityindex.org/v1/companies?api-key={api_key}"

pipeline = ETLPipeline(api_key)
pipeline.run(industry_url, search_url, companies_url)

Data saved to transformed_industries_summary.csv
Data saved to transformed_search_summary.csv
Data saved to cleaned_companies_data.csv
Data saved to cleaned_targets_data.csv


# Data Loading

In [65]:
!pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 320.5 kB/s eta 0:00:00


In [69]:
!pip install psycopg2-binary

     ---------------------------------------- 1.2/1.2 MB 308.7 kB/s eta 0:00:00


ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\hp\\anaconda3\\Lib\\site-packages\\psycopg2\\_psycopg.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [66]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Define database connection parameters
DB_USER = 'your_username'
DB_PASSWORD = 'your_password'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'sustainability_data'

# Create a connection to the PostgreSQL database
def create_connection():
    conn_str = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    engine = create_engine(conn_str)
    return engine

In [67]:
# Define schema creation function
def create_schema(engine):
    with engine.connect() as conn:
        conn.execute('''
        CREATE TABLE IF NOT EXISTS industries (
            id SERIAL PRIMARY KEY,
            name VARCHAR,
            slug VARCHAR,
            company_count INTEGER
        )
        ''')

        conn.execute('''
        CREATE TABLE IF NOT EXISTS search (
            id SERIAL PRIMARY KEY,
            name VARCHAR,
            slug VARCHAR,
            type VARCHAR
        )
        ''')

        conn.execute('''
        CREATE TABLE IF NOT EXISTS companies (
            id SERIAL PRIMARY KEY,
            company_name VARCHAR,
            industry VARCHAR,
            commitment_type VARCHAR,
            status VARCHAR,
            commitment_deadline DATE,
            total_reported_emission_scope_1_2_3 REAL,
            revenue REAL,
            hq_country_move VARCHAR,
            year INTEGER,
            currency VARCHAR,
            emission_intensity REAL
        )
        ''')

        conn.execute('''
        CREATE TABLE IF NOT EXISTS targets (
            id SERIAL PRIMARY KEY,
            lei VARCHAR,
            type VARCHAR,
            scope VARCHAR,
            action VARCHAR,
            target_year INTEGER,
            company_name VARCHAR,
            target_value VARCHAR,
            date_published DATE
        )
        ''')

        conn.execute('''
        CREATE TABLE IF NOT EXISTS emissions (
            id SERIAL PRIMARY KEY,
            year INTEGER,
            status VARCHAR,
            revenue REAL,
            scope_1 REAL,
            currency VARCHAR,
            fiscal_year VARCHAR,
            company_name VARCHAR,
            total_scope_3 VARCHAR,
            cradle_to_gate REAL,
            hq_country_move VARCHAR,
            scope_2_location_based VARCHAR,
            source_emission_link VARCHAR,
            source_emission_report VARCHAR,
            total_reported_emission_scope_1_2 REAL,
            total_reported_emission_scope_1_2_3 REAL
        )
        ''')

In [ ]:
# Load data into PostgreSQL
def load_data_to_db(df, table_name, engine):
    df.to_sql(table_name, engine, if_exists='replace', index=False)

def main():
    # Create connection engine
    engine = create_connection()
    
    # Create schema
    create_schema(engine)
    
    # Load data into tables
    df_industry = pd.read_csv('cleaned_industries_data.csv')
    load_data_to_db(df_industry, 'industries', engine)

    df_search = pd.read_csv('cleaned_data_search.csv')
    load_data_to_db(df_search, 'search', engine)

    df_companies = pd.read_csv('cleaned_companies_data.csv')
    load_data_to_db(df_companies, 'companies', engine)

    # Assuming you have saved targets data separately
    df_targets = pd.read_csv('cleaned_targets_data.csv')
    load_data_to_db(df_targets, 'targets', engine)


if __name__ == '__main__':
    main()